In [1]:
#  Aula Prática – Integração entre PostgreSQL (Pagila) e APIs com Python

# Nesta atividade, você irá:
# - Consultar dados da base Pagila com `psycopg2`
# - Integrar dados climáticos e populacionais usando APIs externas
# - Analisar, transformar e visualizar os dados
# - Praticar o uso de Pandas, APIs, SQL e operações avançadas como `lambda`, `groupby`, `merge`, entre outras

In [2]:
import os

import matplotlib.pyplot as plt
import pandas as pd
import psycopg2
import requests
import seaborn as sns
import time
from dotenv import load_dotenv
from concurrent.futures import ThreadPoolExecutor, as_completed
from tabulate import tabulate

In [ ]:
# Carregar variáveis de ambiente
load_dotenv()

# String de conexão
user = os.getenv("PG_USER")
password = os.getenv("PG_PASSWORD")
host = os.getenv("PG_HOST")
db = os.getenv("PG_DB")
port = os.getenv("PG_PORT")
api_key = os.getenv("WEATHER_KEY")

# Conexão com o banco de dados
engine = psycopg2.connect(
    f"postgresql://{user}:{password}@{host}:{port}/{db}?sslmode=require"
)

True

In [ ]:
def obter_clima_cidade(cidade):
    """
    Obtém a temperatura atual de uma cidade usando a API do WeatherAPI.

    Args:
        cidade (str): Nome da cidade.

    Returns:
        float or None: Temperatura atual em graus Celsius, ou None se não encontrado.
    """
    try:
        url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={cidade}"
        resposta = requests.get(url, timeout=10)
        dados = resposta.json()
        if "current" not in dados:
            # print(f"Clima não encontrado para: {cidade} | Resposta: {dados.get('error', dados)}")
            return None
        else:
            return dados["current"]["temp_c"]
    except Exception as e:
        print(f"Error fetching weather data: {e}")
        return None


# Exemplo de teste
cidade = "Belo Horizonte"
print(f"A temperatura atual em {cidade} é de {obter_clima_cidade(cidade)}°C")

A temperatura atual em Belo Horizonte é de 24.0°C


In [ ]:
def temperatura_cidades(df_cidades):
    """
    Função para obter a temperatura de várias cidades.

    Args:
        df_cidades (pd.DataFrame): DataFrame com as cidades.

    Returns:
        pd.DataFrame: DataFrame com as cidades e suas respectivas temperaturas.
    """
    
    amostra = df_cidades["cidades"].drop_duplicates()
    df_clima = pd.DataFrame({"cidade": amostra})
    df_clima["temperatura"] = df_clima["cidade"].apply(obter_clima_cidade)

    df_analise = df_cidades.merge(
        df_clima, how="left", left_on="cidades", right_on="cidade"
    )
    df_analise = df_analise.drop(columns=["cidade"])
    return df_analise

In [ ]:
def run_query(sql, coon=engine):
    """
    Executa uma consulta SQL na conexão de banco de dados fornecida e retorna o resultado como um DataFrame.

    Args:
        sql (str): Consulta SQL a ser executada.
        coon: Objeto de conexão do banco de dados (por padrão, usa 'engine').

    Returns:
        pd.DataFrame: DataFrame contendo os resultados da consulta SQL.
    """
    return pd.read_sql_query(sql, coon)

In [ ]:
def fetch_client_data():
    q_customer = """
    SELECT 
        c.customer_id, 
        c.first_name, 
        c.last_name, 
        ci.city, 
        co.country
    FROM customer c
    JOIN address a  ON c.address_id = a.address_id
    JOIN city ci    ON a.city_id = ci.city_id
    JOIN country co ON ci.country_id = co.country_id
    """

    df_clientes = run_query(q_customer)
    display(df_clientes.head())

    return df_clientes


df_clientes = fetch_client_data()

C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


,customer_id,first_name,last_name,city,country
0,1,MARY,SMITH,Sasebo,Japan
1,2,PATRICIA,JOHNSON,San Bernardino,United States
2,3,LINDA,WILLIAMS,Athenai,Greece
3,4,BARBARA,JONES,Myingyan,Myanmar
4,5,ELIZABETH,BROWN,Nantou,Taiwan


### Exercício 1 – Temperatura Média das Capitais dos Clientes

- Recupere as cidades dos clientes com mais de 10 transações.
- Use a WeatherAPI para buscar a temperatura atual dessas cidades.
- Calcule a temperatura média ponderada por número de clientes.
- Insight esperado: quais cidades concentram clientes e temperaturas extremas?


In [10]:
# Recupere as cidades dos clientes com mais de 10 transações.
# ---------------

# Query
q_cidades = """
SELECT 
    ci.city as cidades, 
    COUNT(p.payment_id) AS num_transacoes
FROM payment p
    JOIN customer c ON p.customer_id = c.customer_id
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
GROUP BY ci.city
HAVING COUNT(p.payment_id) > 10
ORDER BY num_transacoes DESC
LIMIT 10;
"""

# Executa a query e armazena o resultado em um DataFrame
df_cidades = run_query(q_cidades)

# Exibe o DataFrame
print("As 10 cidades com mais de 10 transações:")
print(tabulate(df_cidades, headers="keys", tablefmt="pretty"))

As 10 cidades com mais de 10 transações:
+---+------------------+----------------+
|   |     cidades      | num_transacoes |
+---+------------------+----------------+
| 0 |     Springs      |       58       |
| 1 |      Aurora      |       50       |
| 2 |      London      |       48       |
| 3 |   Saint-Denis    |       46       |
| 4 |    Cape Coral    |       45       |
| 5 |      Tanza       |       42       |
| 6 |    Molodetno     |       42       |
| 7 |     Changhwa     |       41       |
| 8 |    Changzhou     |       40       |
| 9 | Ourense (Orense) |       40       |
+---+------------------+----------------+


C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


In [ ]:
# Use a WeatherAPI para buscar a temperatura atual dessas cidades.
# ---------------

# Cria um DataFrame com as cidades
df_analise = temperatura_cidades(df_cidades)

# Exibe o DataFrame com as temperaturas
print("Temperaturas atuais das cidades com mais de 10 transações:")
print(tabulate(df_analise, headers="keys", tablefmt="pretty"))

Temperaturas atuais das cidades com mais de 10 transações:
+---+------------------+----------------+-------------+
|   |     cidades      | num_transacoes | temperatura |
+---+------------------+----------------+-------------+
| 0 |     Springs      |       58       |    12.2     |
| 1 |      Aurora      |       50       |    14.6     |
| 2 |      London      |       48       |    13.1     |
| 3 |   Saint-Denis    |       46       |    22.1     |
| 4 |    Cape Coral    |       45       |    32.2     |
| 5 |      Tanza       |       42       |    27.2     |
| 6 |    Molodetno     |       42       |     nan     |
| 7 |     Changhwa     |       41       |    21.1     |
| 8 |    Changzhou     |       40       |    22.9     |
| 9 | Ourense (Orense) |       40       |    11.0     |
+---+------------------+----------------+-------------+


In [12]:
# Calcule a temperatura média ponderada por número de clientes.
# ---------------

# Query
q_cidades = """
WITH top_cidades AS (
    SELECT 
        ci.city as cidades
    FROM payment p
        JOIN customer c ON p.customer_id = c.customer_id
        JOIN address a ON c.address_id = a.address_id
        JOIN city ci ON a.city_id = ci.city_id
    GROUP BY ci.city
    HAVING COUNT(p.payment_id) > 10
    ORDER BY COUNT(p.payment_id) DESC
    LIMIT 10
)
SELECT 
    ci.city as cidades, 
    COUNT(DISTINCT c.customer_id) AS num_clientes
FROM customer c
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
WHERE ci.city IN (SELECT cidades FROM top_cidades)
GROUP BY ci.city
"""

# Executa a query e armazena o resultado em um DataFrame
df_cidades = run_query(q_cidades)

# Atualiza df_analise com as temperaturas dessas cidades
df_analise = temperatura_cidades(df_cidades)

# Converter temperatura para float, se necessário
df_analise["temperatura"] = pd.to_numeric(df_analise["temperatura"], errors="coerce")


# Para calcular a temperatura média ponderada por número de clientes,
# basta multiplicar a temperatura de cada cidade pelo número de clientes,
# somar tudo e dividir pelo total de clientes.
media_ponderada = (
    df_analise["temperatura"] * df_analise["num_clientes"]
).sum() / df_analise["num_clientes"].sum()

# Exibir o resultado ponderado
print(f"Temperatura média ponderada por número de clientes: {media_ponderada:.2f}°C")

C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


Temperatura média ponderada por número de clientes: 16.64°C


In [13]:
# OPCIONAL:

# 1. Pegue as 10 cidades com mais transações
q_cidades = """
SELECT 
    ci.city as cidades, 
    COUNT(p.payment_id) AS num_transacoes
FROM payment p
    JOIN customer c ON p.customer_id = c.customer_id
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
GROUP BY ci.city
HAVING COUNT(p.payment_id) > 10
ORDER BY num_transacoes DESC
LIMIT 10;
"""
df_cidades = run_query(q_cidades)

# 2. Para essas cidades, busque o número de clientes distintos
cidades_sql = ", ".join(f"'{cidade}'" for cidade in df_cidades["cidades"])
q_clientes = f"""
SELECT 
    ci.city as cidades, 
    COUNT(DISTINCT c.customer_id) AS num_clientes
FROM customer c
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
WHERE ci.city IN ({cidades_sql})
GROUP BY ci.city
"""
df_clientes = run_query(q_clientes)

# 3. Junte os dados
df_analise = df_cidades.merge(df_clientes, on="cidades")

# 4. Busque as temperaturas
df_analise = temperatura_cidades(df_analise)
df_analise["temperatura"] = pd.to_numeric(df_analise["temperatura"], errors="coerce")

# 5. Calcule a média ponderada por clientes
media_ponderada = (
    df_analise["temperatura"] * df_analise["num_clientes"]
).sum() / df_analise["num_clientes"].sum()

print(f"Temperatura média ponderada por número de clientes: {media_ponderada:.2f}°C")

C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


Temperatura média ponderada por número de clientes: 16.64°C


#### 🌡️ Temperatura Média Ponderada

A **temperatura média ponderada** por número de clientes (ou transações) é calculada pela fórmula:

**T_ponderada = (T₁ × P₁ + T₂ × P₂ + ... + Tₙ × Pₙ) / (P₁ + P₂ + ... + Pₙ)**

Onde:

- **Tᵢ**: Temperatura da cidade _i_
- **Pᵢ**: Peso da cidade _i_ (número de clientes ou transações)
- **n**: Número total de cidades


In [14]:
# Insight esperado: quais cidades concentram clientes e temperaturas extremas?

# **Explicação da Query:**

# A consulta SQL abaixo tem como objetivo identificar as cidades que concentram o maior número de clientes e transações na base de dados Pagila. Para isso, ela parte da tabela de pagamentos (`payment`), que representa cada transação realizada, e faz uma série de junções para chegar até a cidade de cada cliente. O agrupamento é feito por: cidade, permitindo contar tanto o número total de transações (`num_transacoes`) quanto o número de clientes distintos (`num_clientes`) em cada localidade. Por fim, a query ordena as cidades pelo volume de transações (da maior para a menor) e limita o resultado às 10 cidades mais movimentadas.

# Essa abordagem garante que apenas cidades com movimentação real (ou seja, com pelo menos uma transação registrada) sejam consideradas na análise. Assim, é possível cruzar esses dados com informações de clima e identificar, por exemplo, quais cidades com temperaturas extremas concentram mais clientes e vendas.


# Ordena pelas cidades com mais transações/clientes e temperatura mais alta (limite de 10)
q_cidades = """
WITH top_cidades AS (
    SELECT 
        ci.city as cidades
    FROM payment p
        JOIN customer c ON p.customer_id = c.customer_id
        JOIN address a ON c.address_id = a.address_id
        JOIN city ci ON a.city_id = ci.city_id
    GROUP BY ci.city
    HAVING COUNT(p.payment_id) > 10
    ORDER BY COUNT(p.payment_id) DESC
    LIMIT 10
)
SELECT 
    ci.city as cidades, 
    COUNT(DISTINCT c.customer_id) AS num_clientes
FROM customer c
    JOIN address a ON c.address_id = a.address_id
    JOIN city ci ON a.city_id = ci.city_id
WHERE ci.city IN (SELECT cidades FROM top_cidades)
GROUP BY ci.city
"""

df_cidades = run_query(q_cidades)
df_analise = temperatura_cidades(df_cidades)


# Converter temperatura para float
df_analise["temperatura"] = pd.to_numeric(df_analise["temperatura"], errors="coerce")
# Top 10 cidades mais quentes com mais clientes:
top_quentes = df_analise.sort_values(
    by=["num_clientes", "temperatura"], ascending=[False, False]
)
print("Top 10 cidades mais quentes com mais clientes:")
print(
    tabulate(
        top_quentes[["cidades", "num_clientes", "temperatura"]],
        headers="keys",
        tablefmt="pretty",
    )
)

C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


Top 10 cidades mais quentes com mais clientes:
+---+------------------+--------------+-------------+
|   |     cidades      | num_clientes | temperatura |
+---+------------------+--------------+-------------+
| 0 |      Aurora      |      2       |    14.6     |
| 4 |      London      |      2       |    13.1     |
| 8 |     Springs      |      2       |    12.2     |
| 1 |    Cape Coral    |      1       |    32.2     |
| 9 |      Tanza       |      1       |    27.2     |
| 3 |    Changzhou     |      1       |    22.9     |
| 7 |   Saint-Denis    |      1       |    22.1     |
| 2 |     Changhwa     |      1       |    21.1     |
| 6 | Ourense (Orense) |      1       |    11.0     |
| 5 |    Molodetno     |      1       |     nan     |
+---+------------------+--------------+-------------+


### Exercício 2 – Receita Bruta em Cidades com Clima Ameno

- Calcule a receita bruta por cidade.
- Use a WeatherAPI para consultar a temperatura atual.
- Filtre apenas cidades com temperatura entre **18°C e 24°C**.
- Resultado: qual o faturamento total vindo dessas cidades?


In [15]:
# – Calcule a receita bruta por cidade.
# ---------------


# Query
q_faturamento = f"""
select
    CI.city as cidades,
    SUM(P.amount) as total_faturamento
FROM payment P
    JOIN customer C ON P.customer_id = C.customer_id
    JOIN address A ON C.address_id = A.address_id
    JOIN city CI ON A.city_id = CI.city_id
GROUP BY CI.city
ORDER BY total_faturamento DESC
LIMIT 10;
"""

# Executa a query e armazena o resultado em um DataFrame
df_faturamento = run_query(q_faturamento)

# Exibe o DataFrame
print("Faturamento total das cidades em ordem de faturamento:")
print(tabulate(df_faturamento, headers="keys", tablefmt="pretty"))

Faturamento total das cidades em ordem de faturamento:
+---+---------------------+-------------------+
|   |       cidades       | total_faturamento |
+---+---------------------+-------------------+
| 0 |       Springs       |      243.42       |
| 1 |     Cape Coral      |      221.55       |
| 2 |     Saint-Denis     |      216.54       |
| 3 |       Aurora        |       198.5       |
| 4 |      Molodetno      |      195.58       |
| 5 |      Apeldoorn      |      194.61       |
| 6 | Santa Brbara dOeste |      194.61       |
| 7 |       Qomsheh       |      186.62       |
| 8 |       London        |      180.52       |
| 9 |  Ourense (Orense)   |       177.6       |
+---+---------------------+-------------------+


C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


In [16]:
# 	Use a WeatherAPI para consultar a temperatura atual.
# ---------------

df_analise = temperatura_cidades(df_faturamento)

print("Temperaturas atuais das cidades com faturamento:")
print(tabulate(df_analise, headers="keys", tablefmt="pretty"))

Temperaturas atuais das cidades com faturamento:
+---+---------------------+-------------------+-------------+
|   |       cidades       | total_faturamento | temperatura |
+---+---------------------+-------------------+-------------+
| 0 |       Springs       |      243.42       |    12.2     |
| 1 |     Cape Coral      |      221.55       |    32.2     |
| 2 |     Saint-Denis     |      216.54       |    22.1     |
| 3 |       Aurora        |       198.5       |    14.6     |
| 4 |      Molodetno      |      195.58       |     nan     |
| 5 |      Apeldoorn      |      194.61       |    18.3     |
| 6 | Santa Brbara dOeste |      194.61       |    37.1     |
| 7 |       Qomsheh       |      186.62       |    25.3     |
| 8 |       London        |      180.52       |    13.1     |
| 9 |  Ourense (Orense)   |       177.6       |    11.0     |
+---+---------------------+-------------------+-------------+


In [ ]:
# 	Filtre apenas cidades com temperatura entre 18°C e 24°C.
# 	Resultado: qual o faturamento total vindo dessas cidades?
# ---------------

# Converter para float antes de filtrar
df_analise["temperatura"] = pd.to_numeric(df_analise["temperatura"], errors="coerce")

# Filtrar corretamente
df_filtrado = df_analise.loc[
    (df_analise["temperatura"] >= 18) & (df_analise["temperatura"] <= 24)
]

# Imprimir as cidades com temperatura entre 18 e 24 graus Celsius
# e o faturamento total vindo dessas cidades
print("Imprime as cidades com temperatura entre 18 e 24 graus Celsius:")
print(tabulate(df_filtrado, headers="keys", tablefmt="pretty"))

Imprime as cidades com temperatura entre 18 e 24 graus Celsius:
+---+-------------+-------------------+-------------+
|   |   cidades   | total_faturamento | temperatura |
+---+-------------+-------------------+-------------+
| 2 | Saint-Denis |      216.54       |    22.1     |
| 5 |  Apeldoorn  |      194.61       |    18.3     |
+---+-------------+-------------------+-------------+


### Exercício 3 – Aluguel de Filmes por Região e População

- Identifique os países dos clientes com maior número de aluguéis.
- Use a REST Countries API para obter a população desses países.
- Calcule o número de aluguéis por 1.000 habitantes.
- Análise: quais países são mais “cinéfilos” proporcionalmente?


In [18]:
# Identifique os países dos clientes com maior número de aluguéis.
# ---------------


# Explicação da Query:
# A consulta SQL abaixo tem como objetivo identificar os países que concentram o maior número de aluguéis na base de dados Pagila. Para isso, ela parte da tabela de aluguéis (`rental`) e faz uma série de junções para chegar até o país de cada cliente. O agrupamento é feito por: país, permitindo contar o número total de aluguéis (`num_alugueis`) em cada localidade. Por fim, a query ordena os países pelo volume de aluguéis (da maior para a menor) e limita o resultado aos 10 países mais movimentados.

# Essa abordagem garante que apenas países com movimentação real (ou seja, com pelo menos um aluguel registrado) sejam considerados na análise. Assim, é possível cruzar esses dados com informações de clima e identificar, por exemplo, quais países com temperaturas extremas concentram mais aluguéis e vendas.

# No banco Pagila (baseada no Sakila), os dados estão normalizados (espalhados em várias tabelas):

# rental: cada aluguel (tem rental_id, customer_id, etc)
# customer: quem alugou (tem customer_id, address_id)
# address: endereço do cliente (tem address_id, city_id)
# city: cidade do endereço (tem city_id, country_id)
# country: país da cidade (tem country_id)

# Query
q_alugueis = """
SELECT 
    co.country as pais,                                   -- Seleciona o nome do país
    COUNT(DISTINCT p.rental_id) AS num_alugueis           -- Conta o número de aluguéis únicos por país
FROM rental p
    JOIN inventory i ON p.inventory_id = i.inventory_id   -- (opcional aqui, mas conecta aluguel ao filme)
    JOIN film f ON i.film_id = f.film_id                  -- (opcional aqui, conecta ao filme)
    JOIN customer c ON p.customer_id = c.customer_id      -- Liga aluguel ao cliente
    JOIN address a ON c.address_id = a.address_id         -- Liga cliente ao endereço
    JOIN city ci ON a.city_id = ci.city_id                -- Liga endereço à cidade
    JOIN country co ON ci.country_id = co.country_id      -- Liga cidade ao país
GROUP BY co.country                                       -- Agrupa o resultado por país
ORDER BY num_alugueis DESC                                -- Ordena do país com mais aluguéis para o menor
LIMIT 10;  
"""

# Executa a query e armazena o resultado em um DataFrame
df_alugueis = run_query(q_alugueis)

# Exibe o DataFrame
print("Top 10 países com maior número de aluguéis:")
print(tabulate(df_alugueis, headers="keys", tablefmt="pretty"))

Top 10 países com maior número de aluguéis:
+---+--------------------+--------------+
|   |        pais        | num_alugueis |
+---+--------------------+--------------+
| 0 |       India        |     1572     |
| 1 |       China        |     1426     |
| 2 |   United States    |     968      |
| 3 |       Japan        |     825      |
| 4 |       Mexico       |     763      |
| 5 |       Brazil       |     748      |
| 6 | Russian Federation |     713      |
| 7 |    Philippines     |     568      |
| 8 |       Turkey       |     388      |
| 9 |     Indonesia      |     367      |
+---+--------------------+--------------+


C:\Users\dgeis\AppData\Local\Temp\ipykernel_9732\583913406.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(sql, coon)


In [ ]:
# Use a REST Countries API para obter a população desses países.
# ---------------

# URL da API
url = "https://restcountries.com/v3.1/name/{}"


def obter_populacao_pais(pais):
    """
    Consulta a população de um país usando a REST Countries API.

    Args:
        pais (str): Nome do país a ser consultado.

    Returns:
        int or None: População do país, ou None se não encontrado ou erro.
    """

    try:
        # Monta a URL da API com o nome do país e faz a requisição HTTP
        resposta = requests.get(url.format(pais), timeout=10)
        # Converte a resposta JSON em um dicionário/lista Python
        dados = resposta.json()
        # Verifica se a resposta é uma lista válida e não está vazia
        if isinstance(dados, list) and len(dados) > 0:
            # Retorna o campo 'population' do primeiro país encontrado
            return dados[0]["population"]
        else:
            # Caso não encontre, imprime mensagem de erro e retorna None
            print(f"População não encontrada para: {pais} | Resposta: {dados}")
            return None
    except Exception as e:
        # Em caso de erro na requisição ou parsing, imprime o erro e retorna None
        print(f"Error fetching population data: {e}")
        return None


# Cria um DataFrame com os países
df_alugueis["população"] = df_alugueis["pais"].apply(obter_populacao_pais)

print("População dos países com maior número de aluguéis:")
print(tabulate(df_alugueis, headers="keys", tablefmt="pretty"))

População dos países com maior número de aluguéis:
+---+--------------------+--------------+------------+
|   |        pais        | num_alugueis | população  |
+---+--------------------+--------------+------------+
| 0 |       India        |     1572     | 1380004385 |
| 1 |       China        |     1426     |   649342   |
| 2 |   United States    |     968      |    300     |
| 3 |       Japan        |     825      | 125836021  |
| 4 |       Mexico       |     763      | 128932753  |
| 5 |       Brazil       |     748      | 212559409  |
| 6 | Russian Federation |     713      | 144104080  |
| 7 |    Philippines     |     568      | 109581085  |
| 8 |       Turkey       |     388      |  84339067  |
| 9 |     Indonesia      |     367      | 273523621  |
+---+--------------------+--------------+------------+


In [ ]:
# Calcule o número de aluguéis por 1.000 habitantes.
# ---------------

df_alugueis["alugueis_por_1000"] = (
    (df_alugueis["num_alugueis"] / df_alugueis["população"]) * 1000
).round(4)

# Exibe o DataFrame com a nova coluna
print(tabulate(df_alugueis, headers="keys", tablefmt="pretty"))

+---+--------------------+--------------+------------+-------------------+
|   |        pais        | num_alugueis | população  | alugueis_por_1000 |
+---+--------------------+--------------+------------+-------------------+
| 0 |       India        |     1572     | 1380004385 |      0.0011       |
| 1 |       China        |     1426     |   649342   |      2.1961       |
| 2 |   United States    |     968      |    300     |     3226.6667     |
| 3 |       Japan        |     825      | 125836021  |      0.0066       |
| 4 |       Mexico       |     763      | 128932753  |      0.0059       |
| 5 |       Brazil       |     748      | 212559409  |      0.0035       |
| 6 | Russian Federation |     713      | 144104080  |      0.0049       |
| 7 |    Philippines     |     568      | 109581085  |      0.0052       |
| 8 |       Turkey       |     388      |  84339067  |      0.0046       |
| 9 |     Indonesia      |     367      | 273523621  |      0.0013       |
+---+--------------------

In [ ]:
# Análise: quais países são mais “cinéfilos” proporcionalmente?
# ---------------

# Ordena os países pelo número de aluguéis por 1.000 habitantes (do maior para o menor)
df_cinefilos = df_alugueis.sort_values(by="alugueis_por_1000", ascending=False)

print(
    "Ranking dos países mais 'cinéfilos' proporcionalmente (alugueis por 1.000 habitantes):"
)
print(
    tabulate(
        df_cinefilos[["pais", "num_alugueis", "população", "alugueis_por_1000"]],
        headers="keys",
        tablefmt="pretty",
    )
)

Ranking dos países mais 'cinéfilos' proporcionalmente (alugueis por 1.000 habitantes):
+---+--------------------+--------------+------------+-------------------+
|   |        pais        | num_alugueis | população  | alugueis_por_1000 |
+---+--------------------+--------------+------------+-------------------+
| 2 |   United States    |     968      |    300     |     3226.6667     |
| 1 |       China        |     1426     |   649342   |      2.1961       |
| 3 |       Japan        |     825      | 125836021  |      0.0066       |
| 4 |       Mexico       |     763      | 128932753  |      0.0059       |
| 7 |    Philippines     |     568      | 109581085  |      0.0052       |
| 6 | Russian Federation |     713      | 144104080  |      0.0049       |
| 8 |       Turkey       |     388      |  84339067  |      0.0046       |
| 5 |       Brazil       |     748      | 212559409  |      0.0035       |
| 9 |     Indonesia      |     367      | 273523621  |      0.0013       |
| 0 |       I

In [ ]:
#   Exercício 4 – Filmes Mais Populares em Cidades Poluídas
# 	Liste as 10 cidades com maior número de clientes.
# 	Use a AirVisual API para consultar o AQI dessas cidades.
# 	Relacione os filmes mais alugados em cidades com AQI > 150.
# 	Discussão: poluição impacta preferências de filmes?

In [ ]:
#   Exercício 5 – Clientes em Áreas Críticas
# 	Recupere os clientes com endereço em cidades com AQI acima de 130.
# 	Combine nome do cliente, cidade, país, temperatura e AQI.
# 	Classifique os clientes em “zona de atenção” com base nos critérios ambientais.

In [ ]:
#   Exercício 6 – Receita por Continente
# 	Use a REST Countries API para mapear o continente de cada país.
# 	Agrupe a receita total por continente.
# 	Exiba os resultados em um gráfico de pizza com matplotlib.

In [ ]:
#   Exercício 7 – Tempo Médio de Aluguel vs Clima
# 	Calcule o tempo médio de aluguel por cidade (entre rental_date e return_date).
# 	Combine com a temperatura atual dessas cidades.
# 	Visualize a correlação entre temperatura e tempo médio de aluguel (scatterplot + linha de tendência).

In [ ]:
#   Exercício 8 – Perfil de Clima por Cliente
# 	Para cada cliente, crie um perfil com:
# 	cidade, temperatura, AQI, total de aluguéis, gasto total.
# 	Agrupe os perfis por faixa etária (simulada ou fictícia) e avalie padrões.
# 	Objetivo: conectar comportamento de consumo e ambiente.

In [ ]:
#   Exercício 9 – Exportação Inteligente
# 	Gere um relatório Excel com os seguintes critérios:
# 	Clientes de países com temperatura < 15°C
# 	AQI acima de 100
# 	Receita individual > média geral
# 	Utilize OpenPyXL e organize em múltiplas abas: Clientes, Temperaturas, Alertas.

In [ ]:
#   Exercício 10 – API Cache Inteligente (Desafio)
# 	Implemente uma lógica que salve os dados de clima e AQI localmente em CSV.
# 	Ao consultar novamente a mesma cidade, busque do CSV ao invés da API.
# 	Evite chamadas redundantes — bom para práticas de performance e economia de requisições.